In [1]:
import pyqubo as pq
import numpy as np

from pyqubo import Array
from pyqubo import Binary
import neal

Model the following equation based on Example 1 of the QUBO Tutorial Book (pp.5-6)

$$y = \min{x^TQx} $$
$$ y = -5(x_0)^2 - 3(x_1)^2 - 8(x_2)^2 - 6(x_3)^2 +  4x_0x_1 +8x_0x_2 + 2x_1x_2 + 10x_2x_3$$

## 1. Using Straightforward Binary Quadratic Equations

### Variable Declaration

In [2]:
x0, x1, x2, x3 = Binary('x0'), Binary('x1'), Binary('x2'), Binary('x3')

### Designing the Hamiltonian

In [3]:
linear = -5*x0**2 - 3*x1**2 - 8*x2**2 - 6*x3**2
quadratic = 4*x0*x1 + 8*x0*x2 + 2*x1*x2 + 10*x2*x3

In [5]:
H = linear + quadratic
print(H)

((((10.000000 * Binary('x2')) * Binary('x3')) + ((2.000000 * Binary('x1')) * Binary('x2')) + ((4.000000 * Binary('x0')) * Binary('x1')) + ((8.000000 * Binary('x0')) * Binary('x2'))) + (-1.000000 * (6.000000 * (Binary('x3') * Binary('x3')))) + (-1.000000 * (8.000000 * (Binary('x2') * Binary('x2')))) + (-5.000000 * (Binary('x0') * Binary('x0'))) + (-1.000000 * (3.000000 * (Binary('x1') * Binary('x1')))))


### Model Compilation

In [6]:
model = H.compile()
bqm = model.to_bqm()

### Running Simulated Annealing

In [7]:
sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm, num_reads=1024)

In [8]:
decoded_samples = model.decode_sampleset(sampleset)
best_sample = min(decoded_samples, key=lambda x: x.energy)

print('y = ', best_sample.energy)
print('x = ', best_sample.sample)

y =  -11.0
x =  {'x0': 1, 'x1': 0, 'x2': 0, 'x3': 1}


## 2. Using a more vectorized approach (Arrays)

In [9]:
x = Array.create('x', shape=(4), vartype='BINARY')
x

Array([Binary('x[0]'), Binary('x[1]'), Binary('x[2]'), Binary('x[3]')])

In [10]:
## make a numpy matrix for coeff assignment
Q = np.zeros((4,4))

## Quadratic part
## Interactions are always halved to preserve equivalence to Ising models
Q[0,1] = 4/2
Q[0,2] = 8/2
Q[1,2] = 2/2
Q[2,3] = 10/2

Q += Q.T
np.fill_diagonal(Q, [-5,-3,-8,-6]) ## linear part
Q

array([[-5.,  2.,  4.,  0.],
       [ 2., -3.,  1.,  0.],
       [ 4.,  1., -8.,  5.],
       [ 0.,  0.,  5., -6.]])

In [11]:
H = x.T @ Q @ x

In [12]:
model = H.compile()
bqm = model.to_bqm()

In [13]:
sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm, num_reads=1024)
decoded_samples = model.decode_sampleset(sampleset)
best_sample = min(decoded_samples, key=lambda x: x.energy)

print('y = ', best_sample.energy)
print('x = ', best_sample.sample)

y =  -11.0
x =  {'x[0]': 1, 'x[1]': 0, 'x[2]': 0, 'x[3]': 1}
